In [5]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
import regex
from string import punctuation
from nltk.tokenize import sent_tokenize
import numpy as np

from sklearn.metrics import classification_report, accuracy_score

In [18]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [19]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [20]:
pprint(align_words(true[1], bad[1]))

[('апофеозом', 'опофеозом'),
 ('дня', 'дня'),
 ('для', 'для'),
 ('меня', 'меня'),
 ('сегодня', 'сегодня'),
 ('стала', 'стала'),
 ('фраза', 'фраза'),
 ('услышанная', 'услышанная'),
 ('в', 'в'),
 ('новостях', 'новостях')]


In [22]:
f = open("anna_karenina.txt", "r")
data = f.read()

In [23]:
import csv
corpus = open('corpus_5000.txt', 'w')
reader = csv.reader('lenta-ru-news.csv', delimiter=',', quotechar='"')
for i, line in enumerate(reader):
    if i < 5000: # увеличьте количество текстов тут
        corpus.write(line[2].replace('\xa0', ' ') + '\n')

IndexError: list index out of range

In [24]:
def normalize(text):
    
    normalized_text = [(word.strip(punctuation)) for word \
                                                            in text.lower().split()]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text

In [25]:
corpus = []


norm_sents = [normalize(sent) for sent in sent_tokenize(data, 'russian')]
corpus += norm_sents

In [26]:
corpus

[['annotation',
  '«анна',
  'каренина»',
  'это',
  'сложное',
  'психологически',
  'утонченное',
  'остропроблемное',
  'произведение',
  'насыщенное',
  'приметами',
  'времени'],
 ['л.н',
  'толстой',
  'на',
  'страницах',
  'произведения',
  'показывает',
  'как',
  'рушатся',
  'остатки',
  'патриархального',
  'уклада',
  'жизни',
  'в',
  'россии',
  'под',
  'натиском',
  'буржуазного',
  'прогресса',
  'как',
  'падают',
  'нравы',
  'ослабевают',
  'семейные',
  'устои',
  'вырождается',
  'аристократия'],
 ['роман', 'во', 'многом', 'автобиографичен'],
 ['работая',
  'над',
  'ним',
  'толстой',
  'уяснял',
  'взгляд',
  'на',
  'современность',
  'и',
  'свою',
  'собственную',
  'жизнь'],
 ['лев',
  'николаевич',
  'толстой',
  'часть',
  'первая',
  'часть',
  'вторая',
  'часть',
  'третья',
  'часть',
  'четвертая',
  'часть',
  'пятая',
  'часть',
  'шестая',
  'часть',
  'седьмая',
  'часть',
  'восьмая',
  'librs.net',
  'благодарим',
  'вас',
  'за',
  'использова

In [41]:
vocab = []

for sent in corpus:  
    for word in sent:
        if re.match('[а-я]+', word):
            vocab.append(word)


count_vocab = Counter(vocab)

In [42]:
count_vocab

Counter({'каренина»': 1,
         'это': 2207,
         'сложное': 3,
         'психологически': 1,
         'утонченное': 2,
         'остропроблемное': 1,
         'произведение': 3,
         'насыщенное': 1,
         'приметами': 1,
         'времени': 70,
         'л.н': 1,
         'толстой': 5,
         'на': 3594,
         'страницах': 1,
         'произведения': 3,
         'показывает': 6,
         'как': 2642,
         'рушатся': 1,
         'остатки': 3,
         'патриархального': 1,
         'уклада': 1,
         'жизни': 234,
         'в': 5717,
         'россии': 36,
         'под': 169,
         'натиском': 1,
         'буржуазного': 1,
         'прогресса': 2,
         'падают': 1,
         'нравы': 1,
         'ослабевают': 1,
         'семейные': 9,
         'устои': 1,
         'вырождается': 1,
         'аристократия': 2,
         'роман': 5,
         'во': 287,
         'многом': 3,
         'автобиографичен': 1,
         'работая': 2,
         'над': 152,
       

In [28]:
vocab = set(vocab)

In [29]:
def generate_deletes(word):
    splits = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes = [L + R[1:] for L, R in splits if R]
    return deletes

In [30]:
typo_vocab={}
for word in vocab:
    deletes = generate_deletes(word)
    
    for delete in deletes:
        if delete not in typo_vocab:
            typo_vocab[delete] = [word]
        else:
            typo_vocab[delete].append(word)
                

In [151]:
def correct_typo(typo_word):
    if typo_word in vocab:
        return typo_word 
    elif typo_word in typo_vocab:
        return typo_vocab[typo_word]
    else: 
        for typo_delete in generate_deletes(word):
            if typo_delete in typo_vocab:
                return typo_vocab[typo_delete]
            else: 
                return typo_word
                

            
correct_typo('оччччень')        
        
    
    
        
    

'оччччень'

In [172]:
from itertools import repeat
def predict_mistaken(word):
    if type(correct_typo(word)) == str: 
        return word
    else:
        d = []
        for var in correct_typo(word):

            d.extend([var]* count_vocab[var])

        option = Counter(d).most_common(1)[0][0]
        return option
            
        
    
    

In [173]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], predict[i])
    for pair in word_pairs:
        predicted = cashed.get(pair[1], predict_mistaken(pair[1]))
        cashed[pair[0]] = predicted
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1
        
    if not i % 100:
        print(i)
        

0
100
200
300
400
500
600
700
800
900


In [174]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.6979020979020979
0.06676783004552352
0.2063966866083755


In [10]:
tolstoy = open('anna_karenina.txt').read()

In [11]:
punctuation+= '–'
print(punctuation)
def normalize(text):
    res = regex.findall('[а-я]+|[а-я]*?-[а-я]*', text.lower(), flags=regex.POSIX)
    return(res)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~––


In [12]:
sentences_tolstoy = [['<start>', '<start>'] + normalize(text) + ['<end>', '<end>'] for text in sent_tokenize(tolstoy)]

In [13]:
def ngrammer(tokens, n):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

from collections import Counter
unigrams_tolstoy = Counter()
bigrams_tolstoy = Counter()
trigrams_tolstoy = Counter()

for sentence in sentences_tolstoy:
    unigrams_tolstoy.update(sentence)
    bigrams_tolstoy.update(ngrammer(sentence, n =2))
    trigrams_tolstoy.update(ngrammer(sentence, n = 3))



In [14]:
matrix_tolstoy = np.zeros((len(bigrams_tolstoy), 
                   len(unigrams_tolstoy)))

id2word_tolstoy = list(unigrams_tolstoy)
word2id_tolstoy = {word:i for i, word in enumerate(id2word_tolstoy)}

id2bigram_tolstoy = list(bigrams_tolstoy)
bigram2id_tolstoy = {bigram:i for i, bigram in enumerate(id2bigram_tolstoy)}


#вероятность увидеть слово из униграммы после биграммы: 

for trigram in trigrams_tolstoy:
    word1, word2, word = trigram.split()
    bigram = ''.join(word1+' '+word2)
    matrix_tolstoy[bigram2id_tolstoy[bigram]][word2id_tolstoy[word]] = (trigrams_tolstoy[trigram]/bigrams_tolstoy[bigram])

In [186]:
matrix_tolstoy[[word2id_tolstoy['дом']]]

KeyError: 'пир'

In [ ]:
matrix_tolstoy[[word2id_tolstoy['мир']]]
id2word_tolstoy[word2id_tolstoy[word]]
itemindex = numpy.where(array==6.93824958e-03)

In [168]:
predict= []
for sent in bad:
    predict.append(predict_mistaken(sent.split()))

отсутствие
['отсутствие', 'отсутствие', 'отсутствие', 'отсутствие']
отсутствие
усовершенствование
['усовершенствование']
усовершенствование
работа
['работа', 'работа', 'работа', 'работа', 'работа', 'работа', 'работа', 'работа', 'работа', 'работа', 'работа', 'работа', 'работа', 'работа']
работу
['работа', 'работа', 'работа', 'работа', 'работа', 'работа', 'работа', 'работа', 'работа', 'работа', 'работа', 'работа', 'работа', 'работа', 'работу', 'работу', 'работу', 'работу', 'работу', 'работу', 'работу', 'работу', 'работу', 'работу', 'работу', 'работу', 'работу']
работе
['работа', 'работа', 'работа', 'работа', 'работа', 'работа', 'работа', 'работа', 'работа', 'работа', 'работа', 'работа', 'работа', 'работа', 'работу', 'работу', 'работу', 'работу', 'работу', 'работу', 'работу', 'работу', 'работу', 'работу', 'работу', 'работу', 'работу', 'работе', 'работе', 'работе', 'работе', 'работе', 'работе', 'работе', 'работе', 'работе']
работы
['работа', 'работа', 'работа', 'работа', 'работа', 'работа'

эту
['эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', 'эту', '

редакторы
['редакторы']
редакторы
кем
['кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем']
ком
['кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'кем', 'ком', 'ком', 'ком', 'ком', 'ком']
кем
угол
['угол', 'угол', 'угол', 'угол', 'угол', 'угол', 'угол']
угол
фразы
['фразы', 'фразы', 'фразы', 'фразы', 'фразы', 'фразы', 'фразы', 'фразы', 'фразы']
фразы
стул
['стул', 'стул', 'стул', 'стул', 'стул', 'стул', 'стул', 'стул', 'стул', 'стул', 'стул', 'стул', 'стул', 'стул', 'стул', 'стул', 'стул']
стул
тоска
['тоска', 'тоска', 'тоска', 'тоска']
тоска
как-то
['как-то', 'как-то', 

джон
['джон', 'джон', 'джон']
джон
дождем
['дождем', 'дождем', 'дождем']
дождей
['дождем', 'дождем', 'дождем', 'дождей']
дождем
обой
['обой']
обой
сезона
['сезона']
сезона
если
['если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если', 'если'

басня
['басня']
басня
вообще
['вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще']
вообще
['вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'в

...

тестя
['тестя']
тестя
идете
['идете']
идеею
['идете', 'идеею']
идете
какая-то
['какая-то', 'какая-то', 'какая-то', 'какая-то', 'какая-то', 'какая-то', 'какая-то', 'какая-то', 'какая-то', 'какая-то', 'какая-то', 'какая-то', 'какая-то', 'какая-то', 'какая-то', 'какая-то', 'какая-то', 'какая-то', 'какая-то', 'какая-то']
какая-то
контракте
['контракте']
контракте
вообще
['вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще']
вообще
['вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'в

отдел
['отдел', 'отдел', 'отдел']
отдел
что-то
['что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'что-то', 'ч

вообще
['вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще']
вообще
['вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'воо

фрака
['фрака', 'фрака', 'фрака']
мрака
['фрака', 'фрака', 'фрака', 'мрака', 'мрака']
брака
['фрака', 'фрака', 'фрака', 'мрака', 'мрака', 'брака', 'брака', 'брака', 'брака', 'брака', 'брака']
рамка
['фрака', 'фрака', 'фрака', 'мрака', 'мрака', 'брака', 'брака', 'брака', 'брака', 'брака', 'брака', 'рамка']
брака
некоторые
['некоторые', 'некоторые', 'некоторые', 'некоторые', 'некоторые', 'некоторые', 'некоторые', 'некоторые', 'некоторые', 'некоторые', 'некоторые', 'некоторые', 'некоторые', 'некоторые', 'некоторые', 'некоторые', 'некоторые', 'некоторые', 'некоторые', 'некоторые', 'некоторые', 'некоторые', 'некоторые', 'некоторые']
некоторые
вообще
['вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообще', 'вообщ

когда-то
['когда-то', 'когда-то', 'когда-то', 'когда-то', 'когда-то', 'когда-то', 'когда-то', 'когда-то', 'когда-то', 'когда-то']
когда-то
прямо
['прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо', 'прямо']